In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator, img_to_array, array_to_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.optimizers import SGD, Adam, Adadelta
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import pickle

In [ ]:
data_path = "../input/myrecyclemanagerdataset/Compiled Dataset" 
labels = {}
enum = 0
for item in sorted(os.listdir(data_path)):
    labels[enum] = item
    enum+=1
labels

In [ ]:
DataGen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.1
)

train_datagen_nasnet = DataGen.flow_from_directory(
    data_path,
    batch_size = 16,
    target_size = (331,331),
    class_mode = "categorical",
    subset = "training")
validation_datagen_nasnet = DataGen.flow_from_directory(
    data_path,
    batch_size = 16,
    target_size = (331,331),
    class_mode = "categorical",
    subset = "validation")

In [ ]:
from tensorflow.keras.applications import NASNetLarge

nasnetlarge = NASNetLarge(
    include_top=False,
    weights="imagenet",
    input_shape=(331,331,3),
)

In [ ]:
for layers in nasnetlarge.layers:
    layers.trainable = False
    
base_output = GlobalAveragePooling2D()(nasnetlarge.output)
predictions = Dense(12, activation = "softmax")(base_output)
my_nasnetlarge_model = Model(inputs = nasnetlarge.input, outputs = predictions)

In [ ]:
early_stopping = EarlyStopping(monitor = "val_loss", patience=3)
model_save = ModelCheckpoint("nasnetlarge_model.h5", save_best_only=True, verbose=1)
my_nasnetlarge_model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])
history_nasnetlarge = my_nasnetlarge_model.fit(train_datagen_nasnet, validation_data=validation_datagen_nasnet, epochs=20, 
             steps_per_epoch = train_datagen_nasnet.samples//16,
             callbacks=[early_stopping, model_save])

In [ ]:
with open('./trainHistoryNASNetLarge', 'wb') as file_pi:
    pickle.dump(history_nasnetlarge.history, file_pi)